In [8]:
!pip install qiskit
!pip install qiskit-optimization

  Using cached symengine-0.11.0-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (1.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 3.9 MB/s eta 0:00:00:00:0100:01
Using cached symengine-0.11.0-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (39.4 MB)
  Attempting uninstall: symengine
    Found existing installation: symengine 0.9.2
    Uninstalling symengine-0.9.2:
      Successfully uninstalled symengine-0.9.2
  Attempting uninstall: qiskit
    Found existing installation: qiskit 0.42.0
    Uninstalling qiskit-0.42.0:
      Successfully uninstalled qiskit-0.42.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
quantum-serverless 0.9.0 requires cloudpickle==2.2.1, but you have cloudpickle 2.2.0 which is incompatible.
quantum-serverless 0.9.0 requires importlib-metadata>=5.2.0, but you have importlib-metadata 5.1.0 whic

In [11]:
from qiskit import Aer
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from docplex.mp.model import Model
from qiskit.utils import algorithm_globals, QuantumInstance

In [13]:
# Configurar una semilla global para la reproducibilidad
algorithm_globals.random_seed = 123

# Lista de amistades
amistades = [["Mario", "Sarah"], ["Mario", "Raúl"], ["Mario", "Ana"], 
             ["Enrique", "Sarah"], ["Enrique", "Raúl"], ["Saúl", "Ana"]]

# Crear modelo de optimización
mdl = Model('vertex_cover')

# Crear variables binarias únicas para cada persona
personas = set(name for sublist in amistades for name in sublist)
x = {name: mdl.binary_var(name=f'x_{name}') for name in personas}

# Añadir restricciones basadas en la lista de amistades
for amigo1, amigo2 in amistades:
    mdl.add_constraint(x[f'x_{amigo1}'] + x[f'x_{amigo2}'] >= 1)

# Función objetivo: minimizar el número de personas contratadas
mdl.minimize(mdl.sum(x[name] for name in x))

# Convertir el problema a un programa cuadrático
qp = QuadraticProgram()
qp.from_docplex(mdl)

# Configurar y ejecutar QAOA
quantum_instance = QuantumInstance(Aer.get_backend('aer_simulator'), seed_simulator=123, seed_transpiler=123)
qaoa = QAOA(optimizer=COBYLA(), quantum_instance=quantum_instance, reps=3)
optimizer = MinimumEigenOptimizer(qaoa)
result = optimizer.solve(qp)

# Mostrar el resultado
print("Solución óptima:", result.x)
print("Personas contratadas:", [name[2:] for name, value in x.items() if result.x[list(x.keys()).index(name)] > 0.5])

Traceback (most recent call last):
  Cell In[13], line 17
    mdl.add_constraint(x[f'x_{amigo1}'] + x[f'x_{amigo2}'] >= 1)
KeyError: 'x_Mario'

Use %tb to get the full traceback.
